In [50]:
import os
import pandas as pd
import plotly.express as px
from sklearn.preprocessing import MinMaxScaler
import plotly.express as px

In [18]:
folder_path = "../data_v2/data/"

files = os.listdir(folder_path)

raw_data = {}

for file in files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_json(file_path, lines=True)
    raw_data[file] = df

In [31]:
print(raw_data.keys())

dict_keys(['artists.jsonl', 'sessions.jsonl', 'tracks.jsonl', 'users.jsonl'])


In [32]:
sessions_df = raw_data['sessions.jsonl']
tracks_df = raw_data['tracks.jsonl']

In [33]:
track_appearance_counts = sessions_df['track_id'].value_counts()
track_appearance_counts = track_appearance_counts.reindex(tracks_df['id'], fill_value=0)
track_appearance_counts.describe()

count    22412.000000
mean        64.433875
std         10.758655
min         23.000000
25%         57.000000
50%         64.000000
75%         72.000000
max        111.000000
Name: count, dtype: float64

# Sessions w pełni pokrywa wszystkie utwory

In [34]:
event_map = {'play': 1, 'skip': -1, 'like': 2, 'advertisement': 0}

In [35]:
popularity_event_map = {'play': 1, 'skip': -1, 'like': 2}
sessions_df['popularity_coefficient'] = sessions_df['event_type'].map(event_map)
popularity_per_track = sessions_df.groupby('track_id')['popularity_coefficient'].sum().reset_index()
popularity_per_track.rename(columns = {'popularity_coefficient':'popularity_from_sessions'}, inplace = True)

In [36]:
tracks_with_popularity = pd.merge(tracks_df, popularity_per_track, left_on='id', right_on='track_id', how='left')
# niepotrzebne ze względu na pełne pokrycie
# tracks_with_popularity['popularity_from_sessions'].fillna(0, inplace=True)

In [38]:
scaler = MinMaxScaler()
tracks_with_popularity[['normalized_popularity','normalized_popularity_from_sessions']] = scaler.fit_transform(tracks_with_popularity[['popularity','popularity_from_sessions']])

In [39]:
tracks_with_popularity[['normalized_popularity','normalized_popularity_from_sessions']].describe()

,normalized_popularity,normalized_popularity_from_sessions
count,22412.000000,22412.000000
mean,0.216158,0.378079
std,0.167672,0.122217
min,0.000000,0.000000
25%,0.083333,0.292683
50%,0.187500,0.378049
75%,0.333333,0.451220
max,1.000000,1.000000


In [53]:
fig = px.histogram(tracks_with_popularity, x=['normalized_popularity', 'normalized_popularity_from_sessions'],
                   nbins=50, opacity=0.7, barmode='overlay',
                   labels={'value': 'Normalized Popularity'},
                   title='Distribution of Normalized Popularity Values')
fig.show()